In [14]:
import pandas as pd

# Load the staff data from 'staff.xlsx'
staff_data = pd.read_excel('staff.xlsx')

# Display the first few rows to ensure the data is loaded correctly
print(staff_data.head())

                         Job    Project      Month        Type
0                  سائق لودر  Beni Suef 2015-10-01  Industrial
1              Site Engineer        Tb2 2015-10-01  Commercial
2          Project Manager .        Tb2 2015-10-01  Commercial
3              Site Engineer        Tb2 2015-10-01  Commercial
4  Technical Office Engineer        Tb2 2015-10-01  Commercial


In [15]:
# Group the data by 'Project' and 'Job', and create a temporary DataFrame
grouped_data_with_types = staff_data.groupby(['Project', 'Job']).agg(
    Peak_Number=('Job', 'count')
).reset_index()

# Group the data by 'Job' and create another temporary DataFrame
grouped_data_with_averages = staff_data.groupby('Job').agg(
    Average_Start_Month=('Month', 'min'),
    Average_End_Month=('Month', 'max')
).reset_index()

# Merge the two DataFrames, ensuring the 'Type' column is included
merged_grouped_data = pd.merge(grouped_data_with_types, grouped_data_with_averages, on='Job', how='outer')

# Convert 'Month' columns to datetime for accurate calculations
merged_grouped_data['Average_Start_Month'] = pd.to_datetime(merged_grouped_data['Average_Start_Month'])
merged_grouped_data['Average_End_Month'] = pd.to_datetime(merged_grouped_data['Average_End_Month'])

# Calculate average start and end months relative to project duration (month 0 and last month)
merged_grouped_data['Relative_Start_Month'] = merged_grouped_data['Average_Start_Month'].dt.month - 1
merged_grouped_data['Relative_End_Month'] = merged_grouped_data['Average_End_Month'].dt.month - 1

# Display the results
print("Average start and end months for each job (relative to project duration):")
print(merged_grouped_data)

Average start and end months for each job (relative to project duration):
                          Project                                         Job  \
0          10th of Ramadan Bridge                       Blacksmith Supervisor   
1             Air Defense College                       Blacksmith Supervisor   
2     Alamein Coastal Road Bridge                       Blacksmith Supervisor   
3                       Beni Suef                       Blacksmith Supervisor   
4                           C.F.C                       Blacksmith Supervisor   
...                           ...                                         ...   
5203                   Zed Towers                   Assistant Project Manager   
5204                   Zed Towers  BIM Architecture Technical Office Engineer   
5205                   Zed Towers                         MEP BIM Team Leader   
5206                   Zed Towers                           Projects Engineer   
5207                   Zed Towers  

In [16]:
from datetime import timedelta

# Define the function to create the deployment plan
def create_deployment_plan(project_type, duration_months):

  # Filter data based on project type (using the merged DataFrame)
  filtered_data = merged_grouped_data[merged_grouped_data['Type'] == project_type]

  # Calculate average number of jobs
  average_job_count = filtered_data['Peak_Number'].mean()

  # Adjust start and end months based on user input
  start_month = filtered_data['Relative_Start_Month'].mean() + 1
  end_month = start_month + duration_months - 1

  # Ensure end month doesn't exceed dataset's max
  max_end_month = filtered_data['Relative_End_Month'].max()
  if end_month > max_end_month:
    end_month = max_end_month

  # Calculate actual start and end dates
  start_date = datetime.datetime(2024, 1, 1) + timedelta(months=start_month - 1)
  end_date = start_date + timedelta(months=duration_months)

  # Create and return the deployment plan dictionary
  deployment_plan = {
      'Project_Type': project_type,
      'Start_Month': start_month,
      'End_Month': end_month,
      'Number_of_Jobs': int(average_job_count),
      'Start_Date': start_date.strftime("%Y-%m-%d"),
      'End_Date': end_date.strftime("%Y-%m-%d"),
  }

  return deployment_plan

# Get user input
project_type = input("Enter the project type: ")
duration_months = int(input("Enter the project duration in months: "))

# Create and print the deployment plan
deployment_plan = create_deployment_plan(project_type, duration_months)
print("Deployment Plan for", project_type, "Project:")
print(deployment_plan)

KeyError: 'Type'

In [ ]:
print(merged_grouped_data.to_string())

                                Project                                                     Job  Peak_Number Average_Start_Month Average_End_Month  Relative_Start_Month  Relative_End_Month
0                10th of Ramadan Bridge                                   Blacksmith Supervisor            3          2015-12-01        2023-11-01                    11                  10
1                   Air Defense College                                   Blacksmith Supervisor           17          2015-12-01        2023-11-01                    11                  10
2           Alamein Coastal Road Bridge                                   Blacksmith Supervisor            4          2015-12-01        2023-11-01                    11                  10
3                             Beni Suef                                   Blacksmith Supervisor           28          2015-12-01        2023-11-01                    11                  10
4                                 C.F.C                